#Import Libraries

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Load the real and forged signature datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np

real_path = '/content/drive/MyDrive/dataset/original dataset'  # Corrected "original"
forge_path = '/content/drive/MyDrive/dataset/fraud dataset'

# Check if directories exist
if not os.path.exists(real_path):
    print(f"The directory {real_path} does not exist.")
if not os.path.exists(forge_path):
    print(f"The directory {forge_path} does not exist.")

# Load images if directories are valid
real_images = []
if os.path.exists(real_path):
    for img_name in os.listdir(real_path):
        img = cv2.imread(os.path.join(real_path, img_name), cv2.IMREAD_GRAYSCALE)
        real_images.append(img)
real_images = np.array(real_images, dtype=object)

forge_images = []
if os.path.exists(forge_path):
    for img_name in os.listdir(forge_path):
        img = cv2.imread(os.path.join(forge_path, img_name), cv2.IMREAD_GRAYSCALE)
        forge_images.append(img)
forge_images = np.array(forge_images, dtype=object)


The directory /content/drive/MyDrive/dataset/original dataset does not exist.
The directory /content/drive/MyDrive/dataset/fraud dataset does not exist.


#Spliting the Training and Testing Images

In [ ]:
print("Number of real images:", len(real_images))
print("Number of forged images:", len(forge_images))


Number of real images: 0
Number of forged images: 0


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Verify if images were loaded
if len(real_images) == 0:
    print("No images found in the real images directory.")
if len(forge_images) == 0:
    print("No images found in the forged images directory.")

# Proceed if both lists are not empty
if len(real_images) > 0 and len(forge_images) > 0:
    real_labels = np.zeros(len(real_images))
    forge_labels = np.ones(len(forge_images))

    X = np.concatenate((real_images, forge_images), axis=0)
    y = np.concatenate((real_labels, forge_labels), axis=0)

    # Perform train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    print("Train-test split completed successfully.")
else:
    print("Unable to perform train-test split due to missing images.")


No images found in the real images directory.
No images found in the forged images directory.
Unable to perform train-test split due to missing images.


#Loading the Data and Normalizing it

In [ ]:
import cv2
import numpy as np
import os

# Corrected paths
real_path = '/content/drive/MyDrive/dataset/original dataset'
forge_path = '/content/drive/MyDrive/dataset/fraud dataset'

# Set the image size to 128x128
img_size = (128, 128)

real_images = []
if os.path.exists(real_path):
    for img_name in os.listdir(real_path):
        img = cv2.imread(os.path.join(real_path, img_name), cv2.IMREAD_GRAYSCALE)
        if img is not None:  # Check if the image was loaded successfully
            img = cv2.resize(img, img_size)
            real_images.append(img)
    real_images = np.array(real_images)
else:
    print(f"Directory {real_path} does not exist.")

forge_images = []
if os.path.exists(forge_path):
    for img_name in os.listdir(forge_path):
        img = cv2.imread(os.path.join(forge_path, img_name), cv2.IMREAD_GRAYSCALE)
        if img is not None:  # Check if the image was loaded successfully
            img = cv2.resize(img, img_size)
            forge_images.append(img)
    forge_images = np.array(forge_images)
else:
    print(f"Directory {forge_path} does not exist.")

# Normalize the data if images are loaded
if len(real_images) > 0:
    real_images = real_images.astype('float32') / 255.0
if len(forge_images) > 0:
    forge_images = forge_images.astype('float32') / 255.0


Directory /content/drive/MyDrive/dataset/original dataset does not exist.
Directory /content/drive/MyDrive/dataset/fraud dataset does not exist.


#Create labels for the real and forged signatures

In [ ]:
import numpy as np

num_real_images = len(real_images)
num_forge_images = len(forge_images)

# Create labels for the real and forged signatures
real_labels = np.zeros(num_real_images, dtype=int)
forge_labels = np.ones(num_forge_images, dtype=int)

# Concatenate the real and forged images and labels
X = np.concatenate((real_images, forge_images), axis=0)
y = np.concatenate((real_labels, forge_labels), axis=0)

#Reshaping the Training Set

In [ ]:
import numpy as np

# create dummy data
X_train = np.random.rand(40, 128, 128)

# add another dimension to the array
X_train = np.expand_dims(X_train, axis=-1)

# reshape the array
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)

print(X_train.shape)  # output: (40, 128, 128, 1)

(40, 128, 128, 1)


#Reshaping the Test Set

In [ ]:

import numpy as np

# create dummy data
X_test = np.random.rand(40, 128, 128)

# add another dimension to the array
X_test = np.expand_dims(X_test, axis=-1)

# reshape the array
X_test = X_train.reshape(X_test.shape[0], 128, 128, 1)

print(X_train.shape)  # output: (40, 128, 128, 1)

(40, 128, 128, 1)


#Making the CNN model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Create a Sequential model
model = Sequential()

# Add a convolutional layer
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(128, 128, 1)))

# Add a max pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# Add another convolutional layer
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

# Add another max pooling layer
model.add(MaxPooling2D(pool_size=(2,2)))

# Flatten the output from the convolutional layers
model.add(Flatten())

# Add a fully connected layer with 128 neurons and a relu activation function
model.add(Dense(units=128, activation='relu'))

# Add a dropout layer to reduce overfitting
model.add(Dropout(rate=0.5))

# Add the output layer with a sigmoid activation function
model.add(Dense(units=1, activation='sigmoid'))

# Print a summary of the model architecture
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       7,372,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,391,873 (28.20 MB)

 Trainable params: 7,391,873 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

#Evaluating the Mode

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming real_images and forge_images were defined in previous code cells
real_labels = np.zeros(real_images.shape[0]) if len(real_images) > 0 else []
forge_labels = np.ones(forge_images.shape[0]) if len(forge_images) > 0 else []

# Check if data exists before proceeding
if len(real_labels) > 0 and len(forge_labels) > 0:
    X = np.concatenate((real_images, forge_images), axis=0)
    y = np.concatenate((real_labels, forge_labels), axis=0)

    # Perform train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Compile and train the model if the split was successful
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

    print("Training completed successfully.")
else:
    print("Data is missing. Please ensure real_images and forge_images contain images.")


Data is missing. Please ensure real_images and forge_images contain images.


#Testing Loss and Accuracy

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

# Assume X is your dataset and y are the corresponding labels
X = np.random.rand(1000, 128, 128, 1)  # Example image data
y = np.random.randint(2, size=1000)    # Example binary labels

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Example data preparation
X = np.random.rand(1000, 128, 128, 1)  # Your input data
y = np.random.randint(2, size=1000)    # Your labels

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test))


Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.5057 - loss: 0.7229 - val_accuracy: 0.4850 - val_loss: 0.6976
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.5105 - loss: 0.6943 - val_accuracy: 0.4850 - val_loss: 0.6948
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 39s 960ms/step - accuracy: 0.4886 - loss: 0.6940 - val_accuracy: 0.4850 - val_loss: 0.6960
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 42s 983ms/step - accuracy: 0.5211 - loss: 0.6923 - val_accuracy: 0.4850 - val_loss: 0.6942
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 40s 947ms/step - accuracy: 0.5351 - loss: 0.6928 - val_accuracy: 0.4850 - val_loss: 0.6936
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.5251 - loss: 0.6925 - val_accuracy: 0.4850 - val_loss: 0.6940
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 37s 950ms/step - accuracy: 0.5486 - loss: 0.6915 - val_accuracy: 0.4850 - val_loss: 0.6946
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5451 - loss: 0.6905 - val_accuracy: 0.4850

#Detection of Real and Forged Signature

In [ ]:
import cv2
import numpy as np

# Load a signature image
image_path = '/content/drive/MyDrive/dataset/orininal dataset/agh1_1.jpg'
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Check if the image was loaded successfully
if img is None:
    print(f"Error: Could not load image from path: {image_path}")
else:
    # Resize the image
    img = cv2.resize(img, (128, 128))
    img = np.array(img).reshape(1, 128, 128, 1) / 255.0

    # Predict the class of the signature image
    prediction = model.predict(img)

    if prediction < 0.5:
        print("The signature is real.")
    else:
        print("The signature is forged.")


Error: Could not load image from path: /content/drive/MyDrive/dataset/orininal dataset/agh1_1.jpg
